https://www.codementor.io/jadianes/building-a-recommender-with-apache-spark-python-example-app-part1-du1083qbw

http://www.hongyusu.com/imt/technology/spark-with-python-collaborative-filtering.html

https://datascience.stackexchange.com/questions/14744/spark-als-recommending-for-new-users

streaming new data

https://stackoverflow.com/questions/40809876/apache-spark-als-how-to-perform-live-recommendations-fold-in-anonym-user

In [1]:
import os
import pandas as pd
import math
import findspark

findspark.init('/home/soyoung/standaloneSpark/spark-2.2.1-bin-hadoop2.7')

In [2]:
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating
from pyspark import rdd, SparkContext, SparkFiles


In [5]:
import pyspark

In [ ]:
pyspark.

In [3]:
sc = SparkContext()

In [6]:
data = sc.textFile('ratings.csv')
type(data)

pyspark.rdd.RDD

In [7]:
header = data.first() #extract header
data = data.filter(lambda x: x!=header)

In [10]:
data.take(1)

[u'1,31,2.5,1260759144']

In [11]:
data = data.map(lambda line: line.split(",")).map(lambda tokens: (tokens[0],tokens[1],tokens[2])).cache()

In [12]:
data.take(1)

[(u'1', u'31', u'2.5')]

In [14]:
data.count()

100004

In [13]:
training_RDD, validation_RDD, test_RDD = data.randomSplit([6, 2, 2], seed=0L)

validation_for_predict_RDD = validation_RDD.map(lambda x: (x[0], x[1]))
test_for_predict_RDD = test_RDD.map(lambda x: (x[0], x[1]))

In [16]:
training_RDD.count(), validation_RDD.count(), test_RDD.count()

(60237, 19639, 20128)

In [18]:
validation_RDD.take(1), validation_for_predict_RDD.take(1)

([(u'1', u'1129', u'2.0')], [(u'1', u'1129')])

___

In [19]:
from pyspark.mllib.recommendation import ALS
import math

seed = 7L
iterations = 10
regularization_parameter = 0.1
ranks = [4, 8, 12]
errors = [0, 0, 0]
err = 0
tolerance = 0.02

min_error = float('inf')
best_rank = -1
best_iteration = -1
for rank in ranks:
    model = ALS.train(training_RDD, rank, seed=seed, iterations=iterations,
                      lambda_=regularization_parameter)
    predictions = model.predictAll(validation_for_predict_RDD).map(lambda r: ((r[0], r[1]), r[2]))
    rates_and_preds = validation_RDD.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions)
    error = math.sqrt(rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean())
    errors[err] = error
    err += 1
    print 'For rank %s the RMSE is %s' % (rank, error)
    if error < min_error:
        min_error = error
        best_rank = rank

print 'The best model was trained with rank %s' % best_rank

For rank 4 the RMSE is 0.943466561606
For rank 8 the RMSE is 0.955165016116
For rank 12 the RMSE is 0.95466139835
The best model was trained with rank 4


In [23]:
ALS.train?

In [24]:
%%time
model = ALS.train(training_RDD, rank=4, seed=7, iterations=10, lambda_=0.1)

CPU times: user 20 ms, sys: 4 ms, total: 24 ms
Wall time: 2.71 s


In [39]:
sc.parallelize([(2,3),(4,5)]).map(lambda x: (x[0], x[1])).take(3)

[(2, 3), (4, 5)]

In [45]:
sc.parallelize([(2,3),(4,5)]).take(1)

[(2, 3)]

In [52]:
model.predictAll(sc.parallelize([(0,3),(3,5)])).take(3)

[Rating(user=3, product=5, rating=3.1136962241401553)]

In [62]:
model.recommendUsers(4,10)

[Rating(user=113, product=4, rating=2.791485274086719),
 Rating(user=464, product=4, rating=2.787479362920144),
 Rating(user=156, product=4, rating=2.7695549839540403),
 Rating(user=401, product=4, rating=2.6873789739574416),
 Rating(user=644, product=4, rating=2.682775679056583),
 Rating(user=264, product=4, rating=2.6649385317494323),
 Rating(user=4, product=4, rating=2.6639523027943572),
 Rating(user=530, product=4, rating=2.6119773692133346),
 Rating(user=70, product=4, rating=2.6076697735173364),
 Rating(user=71, product=4, rating=2.604013041300677)]

In [63]:
model.recommendProducts(4,10)

[Rating(user=4, product=83318, rating=7.159958314529181),
 Rating(user=4, product=67504, rating=7.159958314529181),
 Rating(user=4, product=83359, rating=7.159958314529181),
 Rating(user=4, product=759, rating=6.447096149982272),
 Rating(user=4, product=72647, rating=6.443962215234097),
 Rating(user=4, product=25769, rating=6.443962215234097),
 Rating(user=4, product=8797, rating=6.443962215234097),
 Rating(user=4, product=5059, rating=6.443962215234097),
 Rating(user=4, product=7074, rating=6.443962215234097),
 Rating(user=4, product=25764, rating=6.443962215234097)]

In [64]:
ALS?

In [65]:
data.randomSplit?